In [1]:
import pandas as pd
import sqlite3

создаем подключение к базе данных с помощью библиотеки sqlite3

In [2]:
db = sqlite3.connect('../data/checking-logs.sqlite')

## используя только один запрос для каждой из групп, создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками

1. время должно иметь значения: после и до
2. avg_diff содержит среднюю дельту среди всех пользователей за период времени до того, как каждый из них совершил свой первый визит на страницу, и после этого
3. учитывать только тех пользователей, у которых есть наблюдения до и после

In [4]:
test_results_query = """
SELECT
    'before' AS time,
    AVG(CAST((JULIANDAY(datetime(deadlines.deadlines, 'unixepoch')) - JULIANDAY(test.first_commit_ts)) * 24 AS integer)) AS avg_diff
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1' AND test.first_commit_ts < test.first_view_ts
UNION ALL
SELECT
    'after' AS time,
    AVG(CAST((JULIANDAY(datetime(deadlines.deadlines, 'unixepoch')) - JULIANDAY(test.first_commit_ts)) * 24 AS integer)) AS avg_diff
FROM
    test
JOIN
    deadlines
ON
    test.labname = deadlines.labs
WHERE
    test.labname != 'project1' AND test.first_commit_ts >= test.first_view_ts;
"""
test_results = pd.io.sql.read_sql(test_results_query, db)
test_results

,time,avg_diff
0,before,60.56250
1,after,103.40625


In [5]:
control_results_query = """
SELECT
    'before' AS time,
    AVG(CAST((JULIANDAY(datetime(deadlines.deadlines, 'unixepoch')) - JULIANDAY(control.first_commit_ts)) * 24 AS integer)) AS avg_diff
FROM
    control
JOIN
    deadlines
ON
    control.labname = deadlines.labs
WHERE
    control.labname != 'project1' AND control.first_commit_ts < control.first_view_ts
UNION ALL
SELECT 
    'after' AS time,
    AVG(CAST((JULIANDAY(datetime(deadlines.deadlines, 'unixepoch')) - JULIANDAY(control.first_commit_ts)) * 24 AS integer)) AS avg_diff
FROM 
    control
JOIN 
    deadlines
ON
    control.labname = deadlines.labs
WHERE 
    control.labname != 'project1' AND control.first_commit_ts >= control.first_view_ts;
"""
control_result = pd.io.sql.read_sql(control_results_query, db)
control_result

,time,avg_diff
0,before,99.464286
1,after,112.710526


In [6]:
db.close()